<a href="https://colab.research.google.com/github/mukaseevru/ds-school/blob/main/final_project/parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подключение библиотек

In [1]:
# Загрузка необходимых модулей
import numpy as np
import pandas as pd
import datetime as dt
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from time import sleep
from bs4 import BeautifulSoup

# Настройки

In [2]:
# Таймаут, если ошибка соединения
timeout = 1
# Debug mode
debug = True
# Настройки по сайтам
sites = {}
# Включить рассылку
sites['send'] = True
# banki.ru
sites['banki'] = {}
sites['banki']['on'] = True
sites['banki']['first_page'] = 1
sites['banki']['last_page'] = 50
sites['banki']['send'] = True
# vtb.ru
sites['vtb'] = {}
sites['vtb']['on'] = False
sites['vtb']['first_page'] = 1
sites['vtb']['last_page'] = 364
sites['vtb']['send'] = True
# alfabank.ru
sites['alfa'] = {}
sites['alfa']['on'] = True
sites['alfa']['first_page'] = 2021
sites['alfa']['last_page'] = 2022
sites['alfa']['send'] = True

In [3]:
sites

{'alfa': {'first_page': 2021, 'last_page': 2022, 'on': True, 'send': True},
 'banki': {'first_page': 1, 'last_page': 50, 'on': True, 'send': True},
 'send': True,
 'vtb': {'first_page': 1, 'last_page': 364, 'on': False, 'send': True}}

# Определение функций

In [4]:
# Определение функции загрузки страницы
def load_page(url):
  count = 0
  while True:
    try:
      with requests.Session() as session:
        session.headers['Connection'] = 'keep-alive'
        session.headers['Cache-Control'] = 'max-age=0'
        session.headers['Upgrade-Insecure-Requests'] = '1'
        session.headers['User-Agent'] = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.116 Safari/537.36 OPR/40.0.2308.81'
        session.headers['Accept'] = 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'
        session.headers['DNT'] = '1'
        session.headers['Accept-Encoding'] = 'gzip, deflate, lzma, sdch'
        session.headers['Accept-Language'] = 'ru-RU,ru;q=0.8,en-US;q=0.6,en;q=0.4'
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)
        session.proxies = {
            # 'http': '185.198.189.21:8080',
            # 'https': '185.198.189.21:8080'
            }
        response = session.get(url)
      if response.status_code != 200:
        # log.info("Ошибка, код ответа: %s", response.status)
        print('Ошибка, код ответа: %s', response.status_code)
        sleep(timeout)
        count += 1
        if count > 5:
          return (-1)
        # Попробуем снова на следующей итерации цикла
        continue
      # Если дошли до сюда, значит ошибок не было
      return response
    except ConnectionError:
      # Выводим свое сообщение плюс стек трассы
      # log.exception("Ошибка ConnectionError")
      print('Ошибка ConnectionError')
      sleep(timeout)

# Определение функции очистки текста от спецсимволов
def clean_text(text):
  if pd.isnull(text):
    return np.nan
  text = text.replace('\n',' ')\
  .replace('\r',' ')\
  .replace('\t',' ')\
  .replace('\u200b',' ')\
  .replace('  ',' ')\
  .replace('✅',' ')\
  .replace('•',' ')\
  .replace('📰',' ')\
  .replace('🔸',' ')\
  .replace('🔹',' ')\
  .replace('🔸',' ')\
  .replace('🔸',' ')\
  .replace('💰',' ')\
  .replace('👍',' ')\
  .replace('📅',' ')\
  .replace('🍂',' ')\
  .replace('🔥',' ')\
  .replace('🎥',' ')\
  .replace('🎬',' ')\
  .replace('<',' ')\
  .replace('>',' ')\
  .strip()
  while '  ' in text:
    text = text.replace('  ', ' ')
  return text

# banki.ru

## Загрузка имеющихся данных

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os
os.getcwd()

'/content'

In [7]:
banki = pd.read_csv('drive/MyDrive/Colab Notebooks/sber/news/data/banki.csv',
                   dtype={'id': np.str,
                          'title': np.str,
                          'text': np.str,
                          'url': np.str,
                          'time': np.str,
                          'views': np.int64,
                          'comments': np.int64,
                          'source': np.str
                          })
banki.time = pd.to_datetime(banki.time, format='%Y-%m-%d %H:%M:%S')
banki.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113143 entries, 0 to 113142
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   id        113143 non-null  object        
 1   title     113143 non-null  object        
 2   text      113065 non-null  object        
 3   url       113143 non-null  object        
 4   time      113143 non-null  datetime64[ns]
 5   views     113143 non-null  int64         
 6   comments  113143 non-null  int64         
 7   source    113104 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 6.9+ MB


In [8]:
banki.tail()

,id,title,text,url,time,views,comments,source
113138,10940322,Эксперты проанализировали кредитное поведение ...,Россияне в 2020 году стали более взвешенно оце...,https://www.banki.ru/news/lenta/?id=10940322,2021-01-23 16:46:00,9354,4,Banki.ru
113139,10940248,«Национальная Фабрика Ипотеки» снизила первона...,«Национальная Фабрика Ипотеки» (бренд «Ипотека...,https://www.banki.ru/news/lenta/?id=10940248,2021-01-23 15:33:00,4490,0,Banki.ru
113140,10939216,Московский Кредитный Банк предлагает ипотеку в...,Московский Кредитный Банк (МКБ) начал сотрудни...,https://www.banki.ru/news/lenta/?id=10939216,2020-12-27 18:41:00,8265,0,Banki.ru
113141,10939326,Песков: экономика России просела в пандемию ме...,"Решения, которые принимались в России во время...",https://www.banki.ru/news/lenta/?id=10939326,2020-12-27 17:50:00,4474,4,Прайм
113142,10938841,Автоградбанк запустил детские карты,"Автоградбанк приступил к выпуску детских карт,...",https://www.banki.ru/news/lenta/?id=10938841,2020-12-20 17:19:00,7102,0,Banki.ru


## Цикл парсинга

In [9]:
if sites['banki']['on']:
  for page in range(sites['banki']['first_page'], sites['banki']['last_page']):
    count_add_articles = 0
    count_upd_views = 0
    count_upd_comments = 0
    count_upd_source = 0
    url = 'https://www.banki.ru/news/lenta/page' + str(page)
    response = load_page(url)
    if response == -1:
      continue
    soup = BeautifulSoup(response.text, 'lxml')
    articles = soup.find_all('ul', class_='text-list text-list--date text-list--date-inline')[0].find_all('li')
    for article in articles:
      # Кол-во просмотров
      views = 0
      # Кол-во комментариев
      comments = 0
      # Источник новости
      source = ''
      article_url = article.find('a', class_='text-list-link')
      if article_url.get('href')[0] == 'h':
          # print('Error url - {}'.format(article_url.get('href')))
          continue
      article_id = str(article_url.get('href').split('=')[1])
      article_info = article.find_all('span', class_='news__info')
      if len(article_info) > 1:
        source = clean_text(article_info[0].text)
        if article_info[1].text != '':
          views = int(clean_text(article_info[1].text.split('\n\t')[1]))
        if article_info[1].find('span', class_='link-with-icon__icon icon-font icon-bubble-16 icon-font--size_small'):
          comments = int(clean_text(article_info[1].text.split('\n\t')[3]))
      else:
        if article_info[0].text != '':
          views = int(clean_text(article_info[0].text.split('\n\t')[1]))
        if article_info[0].find('span', class_='link-with-icon__icon icon-font icon-bubble-16 icon-font--size_small'):
          comments = int(clean_text(article_info[0].text.split('\n\t')[3]))
      if banki[banki['id'] == article_id].shape[0] == 0:
        url = 'https://www.banki.ru' + article_url.get('href')
        response = load_page(url)
        if response == -1:
          continue
        soup = BeautifulSoup(response.text, 'lxml')
        # Title
        title = soup.find('h1', class_='header-h0')
        # Text
        text = soup.find('article', class_='article-text plain-text markup-inside lenta')
        # Time
        time = soup.find('time')
        banki = banki.append({'id': str(article_id),
                              'title': str(clean_text(title.text)),
                              'text': str(clean_text(text.text)),
                              'url': str(url),
                              'time': pd.to_datetime(time.text, format='%d.%m.%Y %H:%M'),
                              'views': int(views),
                              'comments': int(comments),
                              'source': str(source)}, ignore_index=True)
        count_add_articles += 1
      else:
        if banki.loc[banki['id'] == article_id, 'views'].values[0] != views:
          banki.loc[banki['id'] == article_id, 'views'] = views
          count_upd_views += 1
        if banki.loc[banki['id'] == article_id, 'comments'].values[0] != comments:
          banki.loc[banki['id'] == article_id, 'comments'] = comments
          count_upd_comments += 1
        if banki.loc[banki['id'] == article_id, 'source'].values[0] != source:
          banki.loc[banki['id'] == article_id, 'source'] = source
          count_upd_source += 1
    sites['banki']['first_page'] = page
    if debug:
      print('Page {}: ADD {}, UPD {} views, {} comments, {} source'.format(page, count_add_articles, count_upd_views, count_upd_comments, count_upd_source))

Page 1: ADD 12, UPD 33 views, 4 comments, 0 source
Page 2: ADD 8, UPD 47 views, 2 comments, 0 source
Page 3: ADD 0, UPD 4 views, 0 comments, 0 source
Page 4: ADD 0, UPD 15 views, 0 comments, 0 source
Page 5: ADD 0, UPD 26 views, 0 comments, 1 source
Page 6: ADD 0, UPD 1 views, 0 comments, 0 source
Page 7: ADD 0, UPD 3 views, 0 comments, 0 source
Page 8: ADD 0, UPD 8 views, 0 comments, 0 source
Page 9: ADD 0, UPD 19 views, 0 comments, 0 source
Page 10: ADD 0, UPD 25 views, 0 comments, 0 source
Page 11: ADD 0, UPD 36 views, 0 comments, 0 source
Page 12: ADD 0, UPD 4 views, 0 comments, 0 source
Page 13: ADD 0, UPD 12 views, 0 comments, 0 source
Page 14: ADD 0, UPD 15 views, 0 comments, 0 source
Page 15: ADD 0, UPD 25 views, 0 comments, 0 source
Page 16: ADD 0, UPD 34 views, 0 comments, 0 source
Page 17: ADD 0, UPD 36 views, 0 comments, 0 source
Page 18: ADD 0, UPD 9 views, 0 comments, 0 source
Page 19: ADD 0, UPD 17 views, 0 comments, 0 source
Page 20: ADD 0, UPD 27 views, 0 comments, 0 s

In [10]:
banki.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113171 entries, 0 to 113170
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   id        113171 non-null  object        
 1   title     113171 non-null  object        
 2   text      113093 non-null  object        
 3   url       113171 non-null  object        
 4   time      113171 non-null  datetime64[ns]
 5   views     113171 non-null  int64         
 6   comments  113171 non-null  int64         
 7   source    113133 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 6.9+ MB


## Обработка дубликатов

In [11]:
banki.id.nunique()

113171

In [12]:
banki.drop_duplicates(subset=['id'], inplace=True, keep='last')
banki.shape

(113171, 8)

In [13]:
banki[banki.duplicated()].shape

(0, 8)

## Выгрузка в csv

In [ ]:
banki.to_csv('drive/MyDrive/Colab Notebooks/sber/news/data/banki.csv', index=False)

In [ ]:
banki.head()

Самая просматриваемая статья

In [ ]:
banki[banki.views == banki.views.max()]

## Очистка (убрать комментарии)

In [ ]:
# banki['title'] = banki.title.apply(clean_text)
# banki['text'] = banki.text.apply(clean_text)

# vtb.ru

## Загрузка имеющихся данных

In [18]:
vtb = pd.read_csv('drive/MyDrive/Colab Notebooks/sber/news/data/vtb.csv',
                   dtype={'id': np.str,
                          'title': np.str,
                          'text': np.str,
                          'url': np.str,
                          'time': np.str,
                          'views': np.int64,
                          'comments': np.int64,
                          'source': np.str
                          })
vtb.time = pd.to_datetime(vtb.time, format='%Y-%m-%d %H:%M:%S')
vtb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        2891 non-null   object        
 1   title     2889 non-null   object        
 2   text      2889 non-null   object        
 3   url       2891 non-null   object        
 4   time      2891 non-null   datetime64[ns]
 5   views     2891 non-null   int64         
 6   comments  2891 non-null   int64         
 7   source    2891 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 180.8+ KB


## Цикл парсинга

In [19]:
if sites['vtb']['on']:
  for page in range(sites['vtb']['first_page'], sites['vtb']['last_page']):
    count_add_articles = 0
    url = 'https://vtb.ru/o-banke/press-centr/novosti-i-press-relizy/?page=' + str(page)
    response = load_page(url)
    if response == -1:
      continue
    soup = BeautifulSoup(response.text, 'lxml')
    articles = soup.find_all('div', class_='news-grid__content__item')
    for article in articles:
      # Кол-во просмотров
      views = 0
      # Кол-во комментариев
      comments = 0
      # Источник новости
      source = 'vtb.ru'
      article_url = article.find('a').get('href')
      if article_url == '':
        continue
      article_id = str(article_url.split('/')[6])
      if vtb[vtb['id'] == article_id].shape[0] == 0:
        url = 'https://vtb.ru' + article_url
        response = load_page(url)
        if response == -1:
          continue
        soup = BeautifulSoup(response.text, 'lxml')
        # Title
        title = soup.find('section', class_='common-header').find('h1').text
        # Text
        text = soup.find('div', class_='news-item__main common-text').text
        # Time
        time = soup.find('div', class_='date-block').text.replace('дата','').replace('\n','')
        vtb = vtb.append({'id': str(article_id),
                        'title': str(clean_text(title)),
                        'text': str(clean_text(text)),
                        'url': str(url),
                        'time': pd.to_datetime(time, format='%d.%m.%Y'),
                        'views': int(views),
                        'comments': int(comments),
                        'source': str(source)}, ignore_index=True)
        count_add_articles += 1
    sites['vtb']['first_page'] = page
    if debug:
      print('Page {}: ADD {}'.format(page, count_add_articles))

In [20]:
vtb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        2891 non-null   object        
 1   title     2889 non-null   object        
 2   text      2889 non-null   object        
 3   url       2891 non-null   object        
 4   time      2891 non-null   datetime64[ns]
 5   views     2891 non-null   int64         
 6   comments  2891 non-null   int64         
 7   source    2891 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 180.8+ KB


## Обработка дубликатов

In [21]:
vtb.id.nunique()

2891

In [22]:
vtb.drop_duplicates(subset=['id'], inplace=True, keep='last')
vtb.shape

(2891, 8)

In [23]:
vtb[vtb.duplicated()].shape

(0, 8)

## Выгрузка в csv

In [24]:
vtb.to_csv('drive/MyDrive/Colab Notebooks/sber/news/data/vtb.csv', index=False)

In [25]:
vtb.tail()

,id,title,text,url,time,views,comments,source
2886,2021-02-03-vtb-i-gk-vtb-lizing-protestiruyut-s...,ВТБ и ГК ВТБ Лизинг протестируют сервис подпис...,ВТБ совместно с ГК ВТБ Лизинг планирует в перв...,https://vtb.ru/o-banke/press-centr/novosti-i-p...,2021-02-03,0,0,vtb.ru
2887,2021-02-03-vtb-narastil-portfel-privlechennykh...,ВТБ нарастил портфель привлеченных средств на ...,По итогам 2020 года портфель привлеченных сред...,https://vtb.ru/o-banke/press-centr/novosti-i-p...,2021-02-03,0,0,vtb.ru
2888,2021-02-03-vtb-rynok-roznichnogo-kreditovaniya...,ВТБ: рынок розничного кредитования в России вы...,Кредитный портфель физлиц российских банков ув...,https://vtb.ru/o-banke/press-centr/novosti-i-p...,2021-02-03,0,0,vtb.ru
2889,2021-02-02-vtb-pervym-zapustil-onlayn-servis-p...,ВТБ первым запустил онлайн-сервис подбора сбер...,ВТБ запустил уникальный на рынке онлайн-сервис...,https://vtb.ru/o-banke/press-centr/novosti-i-p...,2021-02-02,0,0,vtb.ru
2890,2021-02-02-vtb-narastil-kreditnyy-portfel-sred...,ВТБ нарастил кредитный портфель среднего и мал...,Кредитный портфель клиентов среднего и малого ...,https://vtb.ru/o-banke/press-centr/novosti-i-p...,2021-02-02,0,0,vtb.ru


# Очистка (убрать комментарии)

In [26]:
# vtb['title'] = vtb.title.apply(clean_text)
# vtb['text'] = vtb.text.apply(clean_text)

# alfabank.ru

## Загрузка имеющихся данных

In [27]:
alfa = pd.read_csv('drive/MyDrive/Colab Notebooks/sber/news/data/alfa.csv',
                   dtype={'id': np.str,
                          'title': np.str,
                          'text': np.str,
                          'url': np.str,
                          'time': np.str,
                          'views': np.int64,
                          'comments': np.int64,
                          'source': np.str
                          })
alfa.time = pd.to_datetime(alfa.time, format='%Y-%m-%d %H:%M:%S')
alfa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4053 entries, 0 to 4052
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        4053 non-null   object        
 1   title     4053 non-null   object        
 2   text      4031 non-null   object        
 3   url       4053 non-null   object        
 4   time      4053 non-null   datetime64[ns]
 5   views     4053 non-null   int64         
 6   comments  4053 non-null   int64         
 7   source    4053 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 253.4+ KB


## Цикл парсинга

In [28]:
if sites['alfa']['on']:
  url_base = 'https://alfabank.ru'
  for url_year in range(sites['alfa']['first_page'], sites['alfa']['last_page']):
    count_add_articles = 0
    for url_month in range(1, 13):
      if url_month < 10:
        url_month = '0' + str(url_month)
      url = url_base + '/press/news/' + str(url_year) + '/' + str(url_month) + '/'
      response = load_page(url)
      if response == -1:
        continue
      soup = BeautifulSoup(response.content, 'lxml')
      articles = soup.find_all('div', class_='news-item')
      if len(articles) == 0:
        continue
      for article in articles:
        # Кол-во просмотров
        views = 0
        # Кол-во комментариев
        comments = 0
        # Источник новости
        source = 'alfabank.ru'
        article_url = article.find('a').get('href')
        article_id = str(article_url.split('/')[3]) + '_' + str(article_url.split('/')[4]) + '_' + str(article_url.split('/')[5]) + '_' + str(article_url.split('/')[6].split('.')[0])
        if alfa[alfa['id'] == article_id].shape[0] == 0:
          # Title
          title = article.find('div', class_='title').text
          # Time
          time = article.find('div', class_='date').text
          url = url_base + article_url
          response = load_page(url)
          if response == -1:
            continue
          soup = BeautifulSoup(response.content, 'lxml')
          article_text = soup.find('div', class_='news-article').find_all('p')
          # Text
          text = ''
          for text_string in article_text[1:]:
            text += text_string.text
          alfa = alfa.append({'id': str(article_id),
                              'title': str(clean_text(title)),
                              'text': str(clean_text(text)),
                              'url': str(url),
                              'time': pd.to_datetime(time, format='%d.%m.%Y'),
                              'views': int(views),
                              'comments': int(comments),
                              'source': str(source)}, ignore_index=True)
          count_add_articles += 1
    sites['alfa']['first_page'] = url_year
    if debug:
      print('Year {}, ADD {}'.format(url_year, count_add_articles))

Year 2021, ADD 0


In [ ]:
alfa.info()

## Обработка дубликатов

In [30]:
alfa.id.nunique()

4053

In [31]:
alfa.drop_duplicates(subset=['id'], inplace=True, keep='last')
alfa.shape

(4053, 8)

In [32]:
alfa[alfa.duplicated()].shape

(0, 8)

## Выгрузка в csv

In [33]:
alfa.to_csv('drive/MyDrive/Colab Notebooks/sber/news/data/alfa.csv', index=False)

In [ ]:
alfa.tail()

# Очистка (убрать комментарии)

In [35]:
# alfa['title'] = alfa.title.apply(clean_text)
# alfa['text'] = alfa.text.apply(clean_text)

# Выгрузка новостей

In [36]:
banki.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113171 entries, 0 to 113170
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   id        113171 non-null  object        
 1   title     113171 non-null  object        
 2   text      113093 non-null  object        
 3   url       113171 non-null  object        
 4   time      113171 non-null  datetime64[ns]
 5   views     113171 non-null  int64         
 6   comments  113171 non-null  int64         
 7   source    113133 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 7.8+ MB


In [37]:
# номер текущей недели
number_of_week = dt.datetime.now().date().isocalendar()[1]

In [38]:
if sites['send']:
  with pd.ExcelWriter('drive/MyDrive/Colab Notebooks/sber/news/data/send.xlsx') as writer:  
    banki_send = pd.DataFrame(columns=banki.columns)
    if sites['banki']['send']:
      for i in range(1, 8):
        banki_send = banki_send.append(banki[(banki.time.dt.isocalendar().year == 2021) &\
                                             (banki.time.dt.isocalendar().week == number_of_week) &\
                                             (banki.time.dt.isocalendar().day == i)].sort_values(by='views', ascending=False).head(3))
      display(banki_send.head())
      banki_send.to_excel(writer, sheet_name='banki')
    elif sites['vtb']['send']:
      vtb_send = vtb[(vtb.time.dt.isocalendar().year == 2021) & (vtb.time.dt.isocalendar().week == number_of_week)].sort_values(by='time', ascending=True)
      display(vtb_send.head())
      vtb_send.to_excel(writer, sheet_name='vtb')
    elif sites['alfa']['send']:
      alfa_send = alfa[(alfa.time.dt.isocalendar().year == 2021) & (alfa.time.dt.isocalendar().week == number_of_week)].sort_values(by='time', ascending=True)
      display(alfa_send.head())
      alfa_send.to_excel(writer, sheet_name='alfa')
    else:
      banki_send.to_excel(writer, sheet_name='clean')

,id,title,text,url,time,views,comments,source
111000,10941147,Банк «Открытие» предлагает сниженную ставку по...,Банк «Открытие» обновил линейку кредитов налич...,https://www.banki.ru/news/lenta/?id=10941147,2021-02-08 10:44:00,15013,3,Banki.ru
111003,10941138,Газпромбанк проводит акцию по кредитам наличными,Газпромбанк проводит акцию и в течение недели ...,https://www.banki.ru/news/lenta/?id=10941138,2021-02-08 10:05:00,12429,0,Banki.ru
111005,10941139,8 февраля. О чем пишут газеты,Экономисты конгресса США указали на проблему п...,https://www.banki.ru/news/lenta/?id=10941139,2021-02-08 09:29:00,5405,0,
112961,10941243,"ЦБ оштрафовал банк «Открытие», «АльфаСтрахован...","ЦБ привлек банк «Открытие», «АльфаСтрахование»...",https://www.banki.ru/news/lenta/?id=10941243,2021-02-09 15:21:00,7599,0,Banki.ru
112972,10941230,СберБанк запустил рефинансирование ипотеки на ...,СберБанк запустил рефинансирование выданных др...,https://www.banki.ru/news/lenta/?id=10941230,2021-02-09 12:02:00,7007,1,Banki.ru
